<a href="https://colab.research.google.com/github/bilalProgTech/btech-nmims/blob/master/NN-DL/Lab-Work/20230922-Lab-7-1-BTech-NNDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d shivamb/netflix-shows
!unzip *.zip

  0% 0.00/1.34M [00:00<?, ?B/s]
100% 1.34M/1.34M [00:00<00:00, 135MB/s]
Archive:  netflix-shows.zip
  inflating: netflix_titles.csv      


In [2]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf

In [3]:
pd.set_option('display.max_colwidth', 300)

In [4]:
data = pd.read_csv('/content/netflix_titles.csv')
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick M...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure","To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war."
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies","In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life."


In [5]:
data = data[['title', 'description', 'listed_in']]
data.head()

,title,description,listed_in
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.",Documentaries
1,Blood & Water,"After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.","International TV Shows, TV Dramas, TV Mysteries"
2,Ganglands,"To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.","Crime TV Shows, International TV Shows, TV Action & Adventure"
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.","Docuseries, Reality TV"
4,Kota Factory,"In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life.","International TV Shows, Romantic TV Shows, TV Comedies"


In [6]:
data['listed_in'] = data['listed_in'].str.split(',')
data.head()

,title,description,listed_in
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.",[Documentaries]
1,Blood & Water,"After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.","[International TV Shows, TV Dramas, TV Mysteries]"
2,Ganglands,"To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.","[Crime TV Shows, International TV Shows, TV Action & Adventure]"
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.","[Docuseries, Reality TV]"
4,Kota Factory,"In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life.","[International TV Shows, Romantic TV Shows, TV Comedies]"


In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def clean_text(text):
    text = re.sub("[^a-zA-Z]"," ",text)
    text = text.lower()
    text = remove_stopwords(text)
    return text

In [9]:
data['clean_description'] = data['description'].apply(lambda x: clean_text(x))
data.head()

,title,description,listed_in,clean_description
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.",[Documentaries],father nears end life filmmaker kirsten johnson stages death inventive comical ways help face inevitable
1,Blood & Water,"After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.","[International TV Shows, TV Dramas, TV Mysteries]",crossing paths party cape town teen sets prove whether private school swimming star sister abducted birth
2,Ganglands,"To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.","[Crime TV Shows, International TV Shows, TV Action & Adventure]",protect family powerful drug lord skilled thief mehdi expert team robbers pulled violent deadly turf war
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.","[Docuseries, Reality TV]",feuds flirtations toilet talk go among incarcerated women orleans justice center new orleans gritty reality series
4,Kota Factory,"In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life.","[International TV Shows, Romantic TV Shows, TV Comedies]",city coaching centers known train india finest collegiate minds earnest unexceptional student friends navigate campus life


In [11]:
from sklearn.preprocessing import MultiLabelBinarizer

In [12]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(data['listed_in'])

y = multilabel_binarizer.transform(data['listed_in'])

In [13]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
y[1].shape

(73,)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train, test, y_train, y_test = train_test_split(data, y, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [19]:
train.shape, test.shape

((7045, 4), (1762, 4))

In [20]:
train_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
train_tokenizer.fit_on_texts(train['clean_description'].values)
train_word_index = train_tokenizer.word_index
print(train_word_index)

{'<OOV>': 1, 'life': 2, 'new': 3, 'young': 4, 'family': 5, 'man': 6, 'world': 7, 'two': 8, 'love': 9, 'woman': 10, 'friends': 11, 'series': 12, 'must': 13, 'one': 14, 'documentary': 15, 'find': 16, 'school': 17, 'old': 18, 'finds': 19, 'home': 20, 'father': 21, 'help': 22, 'three': 23, 'high': 24, 'lives': 25, 'takes': 26, 'group': 27, 'take': 28, 'years': 29, 'city': 30, 'year': 31, 'girl': 32, 'back': 33, 'save': 34, 'team': 35, 'town': 36, 'true': 37, 'teen': 38, 'mother': 39, 'special': 40, 'son': 41, 'get': 42, 'murder': 43, 'time': 44, 'war': 45, 'four': 46, 'daughter': 47, 'death': 48, 'wife': 49, 'crime': 50, 'show': 51, 'becomes': 52, 'past': 53, 'best': 54, 'comedy': 55, 'friend': 56, 'gets': 57, 'story': 58, 'soon': 59, 'boy': 60, 'secret': 61, 'women': 62, 'based': 63, 'mysterious': 64, 'tries': 65, 'couple': 66, 'day': 67, 'stand': 68, 'student': 69, 'first': 70, 'way': 71, 'follows': 72, 'stories': 73, 'make': 74, 'battle': 75, 'behind': 76, 'become': 77, 'fight': 78, 'st

In [21]:
vocab_length = len(train_word_index) + 1
vocab_length

16906

In [22]:
train_sequences = train_tokenizer.texts_to_sequences(train['clean_description'].values)
len(train_sequences)

7045

In [23]:
test_sequences = train_tokenizer.texts_to_sequences(test['clean_description'].values)
len(test_sequences)

1762

In [24]:
longest_sentence = len(max(train_sequences, key=len))
train_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, padding='post', maxlen=longest_sentence, truncating='post')
test_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=longest_sentence, truncating='post')

In [25]:
longest_sentence

27

In [26]:
embedding_dim = 40
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_length, embedding_dim,
                              input_length=longest_sentence),
    tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(y.shape[1], activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 40)            676240    
                                                                 
 lstm (LSTM)                 (None, 27, 100)           56400     
                                                                 
 dropout (Dropout)           (None, 27, 100)           0         
                                                                 
 batch_normalization (Batch  (None, 27, 100)           400       
 Normalization)                                                  
                                                                 
 lstm_1 (LSTM)               (None, 27, 50)            30200     
                                                                 
 batch_normalization_1 (Bat  (None, 27, 50)            200       
 chNormalization)                                       

In [ ]:
history = model.fit(train_padded_seqeunces, y_train, epochs=10,
                    validation_data=(test_padded_seqeunces, y_test))

Epoch 1/10
221/221 [==============================] - 23s 74ms/step - loss: 18.1556 - accuracy: 0.1016 - val_loss: 21.5986 - val_accuracy: 0.1941
Epoch 2/10
221/221 [==============================] - 15s 70ms/step - loss: 99.7756 - accuracy: 0.1023 - val_loss: 142.0793 - val_accuracy: 0.0278
Epoch 3/10
221/221 [==============================] - 14s 63ms/step - loss: 280.1921 - accuracy: 0.0880 - val_loss: 250.0434 - val_accuracy: 0.0516
Epoch 4/10
156/221 [====================>.........] - ETA: 3s - loss: 411.2015 - accuracy: 0.0883

In [ ]:
test_prob = model.predict(test_padded_seqeunces)
t = 0.2
test_pred = (test_prob >= t).astype(int)

In [ ]:
for i in range(10):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k])
    print("Actual genre: ", multilabel_binarizer.inverse_transform(y_test[k].reshape(1, -1))[0])
    print("Predicted tag: ", multilabel_binarizer.inverse_transform(test_pred[k].reshape(1, -1))[0])
    print('\n')

Movie:  Cannabis
Actual genre:  (' International TV Shows', ' TV Dramas', 'Crime TV Shows')
Predicted tag:  ('British TV Shows', 'Dramas')


Movie:  Asphalt Burning
Actual genre:  (' Comedies', ' International Movies', 'Action & Adventure')
Predicted tag:  ('British TV Shows', 'Dramas')


Movie:  Mumbai Cha Raja
Actual genre:  (' Independent Movies', ' International Movies', 'Dramas')
Predicted tag:  ('British TV Shows', 'Dramas')


Movie:  Freaks – You're One of Us
Actual genre:  (' Independent Movies', ' International Movies', 'Dramas')
Predicted tag:  ('British TV Shows', 'Dramas')


Movie:  Masters of the Universe: Revelation
Actual genre:  (' TV Sci-Fi & Fantasy', 'TV Action & Adventure')
Predicted tag:  ('British TV Shows', 'Dramas')


Movie:  15 August
Actual genre:  (' Dramas', ' Independent Movies', 'Comedies')
Predicted tag:  ('Dramas',)


Movie:  White Rabbit Project
Actual genre:  (' International TV Shows', ' Reality TV', 'Docuseries')
Predicted tag:  ('British TV Shows', 